# Different Ways to Build Keras Models

In this notebook we are going to explore how one can build Keras models. We will follow Chollet's book.

## Sequential mode

Let us build a very basic Keras model.

In [4]:
from tensorflow import keras
from tensorflow.keras import layers

model_1 = keras.Sequential([
layers.Dense(64, activation="relu"),
layers.Dense(10, activation="softmax")
])

The same model can be built from an "empty model" and then add the layers. This might be useful if we want to increase the number of layers automatically.

In [7]:
model_2 = keras.Sequential()
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dense(10, activation='softmax'))

It is important to remark that `model_1` and `model_2` are still "empty", because its weights have no values yet. Until you actually call it on some data, or call its `build()` method with an input shape the weights are null.

In [5]:
model_1.weights

[]

In [8]:
model_2.weights

[]

Notice how we got empty lists. In order to get values:

In [9]:
model_1.build(input_shape=(None, 3))
model_1.weights

[<KerasVariable shape=(3, 64), dtype=float32, path=sequential_2/dense_4/kernel>,
 <KerasVariable shape=(64,), dtype=float32, path=sequential_2/dense_4/bias>,
 <KerasVariable shape=(64, 10), dtype=float32, path=sequential_2/dense_5/kernel>,
 <KerasVariable shape=(10,), dtype=float32, path=sequential_2/dense_5/bias>]

After the model is built, you can display its contents via the `summary()` method, which comes in handy for debugging.

In [10]:
model_1.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)

In the `summary()` output appears a name for the model which is generated automatically. We can changed it to make it more "personal". Let us recover `model_2` as an example.

In [11]:
model_2 = keras.Sequential(name='keras_model_with_name')
model_2.add(layers.Dense(64, activation='relu'))
model_2.add(layers.Dense(10, activation='softmax'))
model_2.build(input_shape=(None, 3))
model_2.summary()

Model: "keras_model_with_name"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)